In [ ]:
!pip install -q autogluon
!pip install -q scikit-learn==1.3.2 


In [ ]:
import sklearn
print('scikit-learn version:', sklearn.__version__)

In [4]:
#!sed -i '/_fit_context/d' /usr/local/lib/python3.11/dist-packages/autogluon/tabular/models/tabular_nn/tabular_nn_model.py


sed: can't read /usr/local/lib/python3.11/dist-packages/autogluon/tabular/models/tabular_nn/tabular_nn_model.py: No such file or directory


In [12]:
import os
import glob
import numpy as np
import pandas as pd
import cv2
from sklearn.model_selection import train_test_split
from collections import defaultdict
from autogluon.multimodal import MultiModalPredictor

# Configuration
IMG_SIZE = (64, 64)
DATA_DIR = '/kaggle/input/byu-locating-bacterial-flagellar-motors-2025'
TRAIN_DIR_PATTERN = os.path.join(DATA_DIR, 'train/*/')
TEST_SLICE_PATTERN = os.path.join(DATA_DIR, 'test/*/*.jpg')
LABEL_PATH = os.path.join(DATA_DIR, 'train_labels.csv')

# Load training labels
train_labels = pd.read_csv(LABEL_PATH)

# Load one slice per training tomogram
def load_motor_slices(train_dirs, labels_df):
    images, coords, ids = [], [], []
    for tomo_path in glob.glob(train_dirs):
        tomo_id = os.path.basename(os.path.normpath(tomo_path))
        label_row = labels_df[labels_df['tomo_id'] == tomo_id]
        if label_row.empty:
            continue
        label = label_row.iloc[0]
        z = int(round(label['Motor axis 0']))
        x = label['Motor axis 2']
        y = label['Motor axis 1']
        slice_paths = sorted(glob.glob(os.path.join(tomo_path, '*.jpg')))
        if len(slice_paths) == 0 or z >= len(slice_paths):
            continue
        img = cv2.imread(slice_paths[z], cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, IMG_SIZE)
        img_path = f'/kaggle/working/temp_train/{tomo_id}_slice.jpg'
        cv2.imwrite(img_path, img)
        images.append(img_path)
        coords.append([x, y])  # ✨ No normalization!
        ids.append(tomo_id)
    return images, coords, ids


# Create temporary directory for processed training images
os.makedirs('/kaggle/working/temp_train', exist_ok=True)

# Load and split data
X_paths, y_coords, ids = load_motor_slices(TRAIN_DIR_PATTERN, train_labels)
train_df = pd.DataFrame({
    'image': X_paths,
    'Motor axis 1': [coord[1] for coord in y_coords],  # Y (vertical axis)
    'Motor axis 2': [coord[0] for coord in y_coords],  # X (horizontal axis)
})


In [16]:
train_df = train_df[(train_df['Motor axis 1'] >= 0) & (train_df['Motor axis 2'] >= 0)].reset_index(drop=True)


In [17]:
train_df

,image,Motor axis 1,Motor axis 2
0,/kaggle/working/temp_train/tomo_82d780_slice.jpg,366.0,574.0
1,/kaggle/working/temp_train/tomo_cad74b_slice.jpg,838.0,639.0
2,/kaggle/working/temp_train/tomo_ec607b_slice.jpg,159.0,202.0
3,/kaggle/working/temp_train/tomo_f78e91_slice.jpg,469.0,387.0
4,/kaggle/working/temp_train/tomo_bf1398_slice.jpg,583.0,185.0
...,...,...,...
357,/kaggle/working/temp_train/tomo_e51e5e_slice.jpg,860.0,588.0
358,/kaggle/working/temp_train/tomo_e685b8_slice.jpg,167.0,466.0
359,/kaggle/working/temp_train/tomo_d21396_slice.jpg,218.0,382.0
360,/kaggle/working/temp_train/tomo_2a6091_slice.jpg,492.0,635.0


In [18]:

train_data, val_data = train_test_split(train_df, test_size=0.2, random_state=42)

predictor = MultiModalPredictor(label=['Motor axis 1', 'Motor axis 2'], problem_type='regression')
predictor.fit(train_data, time_limit=1800, hyperparameters={"model.names": ["resnet"]})

# Save predictor
predictor.save('/kaggle/working/motor_locator_autogluon')

# Efficient test loader
def load_and_preprocess_image_to_tmp(path):
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, IMG_SIZE)
    tmp_path = '/kaggle/working/temp_test/' + os.path.basename(path)
    cv2.imwrite(tmp_path, img)
    return tmp_path

# Prepare submission
def group_test_by_tomo(test_pattern):
    grouped = defaultdict(list)
    for path in sorted(glob.glob(test_pattern)):
        tomo_id = os.path.basename(os.path.dirname(path))
        grouped[tomo_id].append(path)
    return grouped

os.makedirs('/kaggle/working/temp_test', exist_ok=True)

def prepare_submission_autogluon(predictor, test_pattern, threshold=120):
    grouped = group_test_by_tomo(test_pattern)
    submission = []

    for tomo_id, slice_paths in grouped.items():
        tmp_test = []
        tmp_paths = []
        for path in slice_paths:
            processed_path = load_and_preprocess_image_to_tmp(path)
            tmp_test.append({'image': processed_path})
            tmp_paths.append(path)

        test_df = pd.DataFrame(tmp_test)
        preds = predictor.predict(test_df).to_numpy()
        preds = preds * np.array([IMG_SIZE[1], IMG_SIZE[0]])

        norms = np.linalg.norm(preds, axis=1)
        best_idx = np.argmin(norms)

        if norms[best_idx] > threshold:
            submission.append([tomo_id, -1, -1, -1])
        else:
            y, x = preds[best_idx]
            submission.append([tomo_id, best_idx, y, x])

    return submission

# Generate submission
submission = prepare_submission_autogluon(predictor, TEST_SLICE_PATTERN)
submission_df = pd.DataFrame(submission, columns=['tomo_id', 'Motor axis 0', 'Motor axis 1', 'Motor axis 2'])

# Validate and cleanup
test_paths = glob.glob(TEST_SLICE_PATTERN)
expected_tomo_ids = set([os.path.basename(os.path.dirname(p)) for p in test_paths])
submission_df = submission_df[submission_df['tomo_id'].isin(expected_tomo_ids)].drop_duplicates(subset=['tomo_id'])

grouped_test = defaultdict(list)
for p in test_paths:
    tomo = os.path.basename(os.path.dirname(p))
    grouped_test[tomo].append(p)

def clip_z(row):
    if row['Motor axis 0'] == -1:
        return -1
    max_idx = len(grouped_test[row['tomo_id']]) - 1
    return int(min(max(row['Motor axis 0'], 0), max_idx))

submission_df['Motor axis 0'] = submission_df.apply(clip_z, axis=1)
submission_df[['Motor axis 1', 'Motor axis 2']] = submission_df[['Motor axis 1', 'Motor axis 2']].round(2)

# Save final submission
submission_df.to_csv('/kaggle/working/submission.csv', index=False)
print("\n✅ Cleaned submission saved. Preview:")
print(submission_df.head())


No path specified. Models will be saved in: "AutogluonModels/ag-20250426_212701"
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.11.11
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Sun Nov 10 10:07:59 UTC 2024
CPU Count:          4
Pytorch Version:    2.5.1+cu124
CUDA Version:       CUDA is not available
Memory Avail:       14.91 GB / 31.35 GB (47.5%)
Disk Space Avail:   19.50 GB / 19.52 GB (99.9%)


ValueError: Provided DataFrame does not contain label column: ['Motor axis 1', 'Motor axis 2']

In [ ]:
submission_df